In [2]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


In [3]:
import openai as client
assistant = client.beta.assistants.create(
  name="Investor Assistant",
  instructions="You help users do research on publicly traded companies and you help them decide if they should buy the stock or not",
  model="gpt-4-1106-preview",
  tools = functions,
)
assistant




Assistant(id='asst_H9kqsdULnfk8Bex7gXzGxxhQ', created_at=1713061125, description=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help them decide if they should buy the stock or not', metadata={}, model='gpt-4-1106-preview', name='Investor Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Ticker symbol of the company'}}, 'required': ['ticker']}), type='function'), FunctionTool(function=FunctionDefinition(nam

In [ ]:
# assistant_id="asst_EqFJN0EqWZq6ouTpchEgDXz0" 

In [4]:
thread=client.beta.threads.create(
  messages=[
    {
      "role":"user",
      "content":"I want to know if the Salesforce stock is a good buy"
    }
  ]
)
thread

Thread(id='thread_VTPAoB52hq9rJlTgQh7ypQRi', created_at=1713061158, metadata={}, object='thread')

In [5]:
thread.id

'thread_VTPAoB52hq9rJlTgQh7ypQRi'

In [6]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
run.id

'run_3fxQVlqZwYgymyRZjFTgTf5Y'

In [7]:
assistant.id

'asst_H9kqsdULnfk8Bex7gXzGxxhQ'

In [8]:
def get_run(run_id, thread_id):
  return client.beta.threads.runs.retrieve(
    run_id=run_id,
    thread_id=thread_id
  )

def send_message(thread_id, content):
  return client.beta.threads.messages.create(
    thread_id=thread_id, role="user", content=content
  )

def get_messages(thread_id):
  messages = client.beta.threads.messages.list(
    thread_id=thread_id
  )

  messages = list(messages)
  print(messages)
  for message in messages:
    print(f"{message.role}:{message.content[0].text.value}")

  print("===========================================")
  messages.reverse()
  print(messages)
  for message in messages:
    print(f"{message.role}:{message.content[0].text.value}")


In [9]:
get_run(run.id, thread.id).status
# get_messages(thread.id)

[Message(id='msg_SA02R9XLP6U4mPpIVLr7Ooi8', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know if the Salesforce stock is a good buy'), type='text')], created_at=1713061158, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VTPAoB52hq9rJlTgQh7ypQRi')]
user:I want to know if the Salesforce stock is a good buy
[Message(id='msg_SA02R9XLP6U4mPpIVLr7Ooi8', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know if the Salesforce stock is a good buy'), type='text')], created_at=1713061158, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VTPAoB52hq9rJlTgQh7ypQRi')]
user:I want to know if the Salesforce stock is a good buy


In [10]:

send_message(thread.id, "Please go ahead")

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_VTPAoB52hq9rJlTgQh7ypQRi while a run run_3fxQVlqZwYgymyRZjFTgTf5Y is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [13]:
run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
run_status

Run(id='run_3fxQVlqZwYgymyRZjFTgTf5Y', assistant_id='asst_H9kqsdULnfk8Bex7gXzGxxhQ', cancelled_at=None, completed_at=None, created_at=1713061187, expires_at=1713061787, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help them decide if they should buy the stock or not', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_ZQ9beZns6oFFr6OFK0qk9ONW', function=Function(arguments='{"company_name":"Salesforce"}', name='get_ticker'), type='function')]), type='submit_tool_outputs'), started_at=1713061187, status='requires_action', thread_id='thread_VTPAoB52hq9rJlTgQh7ypQRi', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type'

In [67]:
get_messages(thread.id)

[Message(id='msg_lk5wwj2gv5YbtS8D2B4JgFqf', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know if the Salesforce stock is a good buy'), type='text')], created_at=1713059296, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BUQoDvLOf5iMufSoT6HSrHoR')]
user:I want to know if the Salesforce stock is a good buy
[Message(id='msg_lk5wwj2gv5YbtS8D2B4JgFqf', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know if the Salesforce stock is a good buy'), type='text')], created_at=1713059296, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_BUQoDvLOf5iMufSoT6HSrHoR')]
user:I want to know if the Salesforce stock is a good buy


In [5]:
# get_ticker({"company_name":"Cloudflare"})
get_income_statement({"ticker":"NET"})


'"{\\"1703980800000\\":{\\"Tax Effect Of Unusual Items\\":-10563000.0,\\"Tax Rate For Calcs\\":0.21,\\"Normalized EBITDA\\":14130000.0,\\"Total Unusual Items\\":-50300000.0,\\"Total Unusual Items Excluding Goodwill\\":-50300000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":-183949000.0,\\"Reconciled Depreciation\\":135820000.0,\\"Reconciled Cost Of Revenue\\":307005000.0,\\"EBITDA\\":-36170000.0,\\"EBIT\\":-171990000.0,\\"Net Interest Income\\":62295000.0,\\"Interest Expense\\":5872000.0,\\"Interest Income\\":68167000.0,\\"Normalized Income\\":-144212000.0,\\"Net Income From Continuing And Discontinued Operation\\":-183949000.0,\\"Total Expenses\\":1482230000.0,\\"Total Operating Income As Reported\\":-185485000.0,\\"Diluted Average Shares\\":333656000.0,\\"Basic Average Shares\\":333656000.0,\\"Diluted EPS\\":-0.55,\\"Basic EPS\\":-0.55,\\"Diluted NI Availto Com Stockholders\\":-183949000.0,\\"Net Income Common Stockholders\\":-183949000.0,\\"Net Income\\":-183949